## A Simple CNN Baseline Using Pretrained ResNet18
Author: Junye Wang (群柴犬@DataTech工作室)

### 导入必要的库工具

In [1]:
import os
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from sklearn.model_selection import train_test_split
from sklearn.metrics import balanced_accuracy_score
from PIL import Image
from tqdm import tqdm
import numpy as np
import random

### 适配竞赛服务器的文件路径

In [2]:
DATASET_PATH = '/kaggle/input/the-1st-datatech-alchemist-cup-public-dataset'
TRAIN_CSV = os.path.join(DATASET_PATH, 'train.csv')
TEST_CSV = os.path.join(DATASET_PATH, 'test.csv')
TRAIN_IMAGES = os.path.join(DATASET_PATH, 'train_images')
TEST_IMAGES = os.path.join(DATASET_PATH, 'test_images')
SUBMISSION_FILE = '/kaggle/working/submission.csv'

# DATASET_PATH = '/kaggle/input/butterfly-image-classification'
# TRAIN_CSV = os.path.join(DATASET_PATH, 'Training_set.csv')
# TEST_CSV = os.path.join(DATASET_PATH, 'Testing_set.csv')
# TRAIN_IMAGES = os.path.join(DATASET_PATH, 'train')
# TEST_IMAGES = os.path.join(DATASET_PATH, 'test')
# SUBMISSION_FILE = '/kaggle/working/submission.csv'

### 定义超参数

In [3]:
BATCH_SIZE = 32
EPOCHS = 100
LEARNING_RATE = 0.001
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

def set_seed(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed()

In [4]:
def load_csv():
    train_df = pd.read_csv(TRAIN_CSV)
    test_df = pd.read_csv(TEST_CSV)
    labels = train_df['label'].unique().tolist()
    label_to_idx = {label: idx for idx, label in enumerate(labels)}
    idx_to_label = {idx: label for label, idx in label_to_idx.items()}
    return train_df, test_df, label_to_idx, idx_to_label

### 自定义数据集类

In [5]:
class ButterflyDataset(Dataset):
    def __init__(self, dataframe, image_dir, label_to_idx=None, transform=None):
        self.dataframe = dataframe
        self.image_dir = image_dir
        self.label_to_idx = label_to_idx
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_name = self.dataframe.iloc[idx, 0]
        img_path = os.path.join(self.image_dir, img_name)
        image = Image.open(img_path).convert('RGB')

        if self.transform:
            image = self.transform(image)

        if self.label_to_idx:
            label = self.dataframe.iloc[idx, 1]
            label = self.label_to_idx[label]
            return image, label
        else:
            return image, img_name

### 数据增强与预处理

In [6]:
import torchvision.transforms.v2
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
train_transform = transforms.Compose([
    # transforms.Resize((256, 256)),
    # transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),  # 随机裁剪
    # transforms.RandomHorizontalFlip(p=0.5),              # 水平翻转
    # transforms.RandomRotation(15),                       # 随机旋转
    # transforms.ToTensor(),
    # transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    transforms.RandomResizedCrop(224, scale=(0.6, 1.0)),
    transforms.v2.RandAugment(num_ops = 3, magnitude = 9),  # RandAugment
    transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomRotation(45),
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),
    transforms.RandomPerspective(distortion_scale=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    transforms.RandomErasing(p=0.5, scale=(0.02, 0.2))  # 随机擦除
])

In [7]:
def rand_bbox(size, lam):
    """ 生成随机裁剪的边界框 """
    W, H = size[2], size[3]
    cut_rat = np.sqrt(1. - lam)
    cut_w = int(W * cut_rat)
    cut_h = int(H * cut_rat)

    cx = np.random.randint(W)
    cy = np.random.randint(H)

    bbx1 = np.clip(cx - cut_w // 2, 0, W)
    bby1 = np.clip(cy - cut_h // 2, 0, H)
    bbx2 = np.clip(cx + cut_w // 2, 0, W)
    bby2 = np.clip(cy + cut_h // 2, 0, H)

    return bbx1, bby1, bbx2, bby2

def cutmix_data(x, y, alpha):
    """ 执行CutMix增强 """
    if alpha > 0:
        lam = np.random.beta(alpha, alpha)
    else:
        lam = 1

    batch_size = x.size()[0]
    index = torch.randperm(batch_size).to(x.device)

    # 生成混合图像
    bbx1, bby1, bbx2, bby2 = rand_bbox(x.size(), lam)
    x[:, :, bbx1:bbx2, bby1:bby2] = x[index, :, bbx1:bbx2, bby1:bby2]
    
    # 调整lambda值（根据实际裁剪区域比例）
    lam = 1 - ((bbx2 - bbx1) * (bby2 - bby1) / (x.size()[-1] * x.size()[-2]))
    
    # 混合标签
    y_a, y_b = y, y[index]
    return x, y_a, y_b, lam

### 载入数据

In [8]:
# 修改数据加载部分
train_df, test_df, label_to_idx, idx_to_label = load_csv()

# 划分训练集和验证集
train_df, val_df = train_test_split(train_df, test_size=0.2, random_state=42, stratify=train_df['label'])

# 创建数据集
train_dataset = ButterflyDataset(train_df, TRAIN_IMAGES, label_to_idx, train_transform)
val_dataset = ButterflyDataset(val_df, TRAIN_IMAGES, label_to_idx, transform)
test_dataset = ButterflyDataset(test_df, TEST_IMAGES, transform=transform)

# 创建数据加载器
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

设计一个 CNN 模型的 class

In [9]:
# class ButterflyCNN(nn.Module):
#     def __init__(self, num_classes):
#         super(ButterflyCNN, self).__init__()
#         self.model = models.resnet18(pretrained=True)
#         self.model.fc = nn.Linear(self.model.fc.in_features, num_classes)
               
#         for param in self.model.parameters():
#             param.requires_grad = False
        
#         for param in self.model.layer4.parameters():
#             param.requires_grad = True
#         for param in self.model.fc.parameters():
#             param.requires_grad = True
            
#         # 添加Dropout层
#         self.model.fc = nn.Sequential(
#             nn.Dropout(0.5),  
#             nn.Linear(self.model.fc.in_features, num_classes)
#         )

#     def forward(self, x):
#         return self.model(x)

### 实例化待训练模型，载入预训练的 Resnet18 卷积神经网络

### 设计 Loss 函数和优化器

In [10]:
import torch.nn as nn
import torchvision.models as models
import types

import torch
import torch.nn as nn
import torchvision.models as models
import types

# 定义CBAM模块（通道注意力和空间注意力）
class CBAM(nn.Module):
    def __init__(self, channel, reduction=16):
        super().__init__()
        # 通道注意力
        self.channel_att = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Conv2d(channel, channel//reduction, 1),
            nn.ReLU(),
            nn.Conv2d(channel//reduction, channel, 1),
            nn.Sigmoid()
        )
        # 空间注意力
        self.spatial_att = nn.Sequential(
            nn.Conv2d(2, 1, 7, padding=3),
            nn.Sigmoid()
        )

    def forward(self, x):
        # 通道注意力
        channel_att = self.channel_att(x)
        x_channel = x * channel_att
        
        # 空间注意力
        avg_out = torch.mean(x_channel, dim=1, keepdim=True)
        max_out, _ = torch.max(x_channel, dim=1, keepdim=True)
        spatial_att = self.spatial_att(torch.cat([avg_out, max_out], dim=1))
        return x_channel * spatial_att

class ButterflyCNN(nn.Module):
    def __init__(self, num_classes, dropout_rate=0.5):
        super(ButterflyCNN, self).__init__()
        self.model = models.resnet50(pretrained=True)
        
        # Unfreeze specific layers
        for name, param in self.model.named_parameters():
            if "layer3" in name or "layer4" in name or "layer2" in name:
                param.requires_grad = True
            else:
                param.requires_grad = False
        
        # Insert CBAM into ResNet blocks
        def insert_attention(block):
            original_forward = block.forward
    
            def new_forward(self, x):
                identity = x
                out = self.conv1(x)
                out = self.bn1(out)
                out = self.relu(out)
                
                out = self.conv2(out)
                out = self.bn2(out)
                
                if hasattr(self, 'attention'):
                    out = self.attention(out)
                
                if self.downsample is not None:
                    identity = self.downsample(x)
                out += identity
                out = self.relu(out)
                return out
    
            channel = block.conv2.out_channels
            block.attention = CBAM(channel)
            block.forward = types.MethodType(new_forward, block)
        
        # Apply CBAM to layer3 and layer4
        for block in self.model.layer3.children():
            if isinstance(block, models.resnet.BasicBlock):
                insert_attention(block)
        for block in self.model.layer4.children():
            if isinstance(block, models.resnet.BasicBlock):
                insert_attention(block)
        
        # Corrected classification head
        self.model.fc = nn.Sequential(
            nn.Linear(2048, 1024),  # Input features from original avgpool + flatten
            nn.BatchNorm1d(1024),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(1024, num_classes)
        )

    def forward(self, x):
        return self.model(x)

In [11]:
model = ButterflyCNN(num_classes=len(label_to_idx)).to(DEVICE)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 214MB/s]


In [12]:
criterion = nn.CrossEntropyLoss()
# optimizer = optim.AdamW(model.parameters(), lr=LEARNING_RATE, weight_decay=1e-4)  
optimizer = torch.optim.Adam([
    {'params': model.model.layer3.parameters(), 'lr': 1e-4},
    {'params': model.model.layer4.parameters(), 'lr': 1e-4},
    {'params': model.model.fc.parameters(), 'lr': 1e-3}
])
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'max', patience=5, factor=0.5) 


### 训练函数 (>_<)

In [13]:
@torch.no_grad()
def evaluate(loader):
    model.eval()
    total_loss = 0
    all_preds = []
    all_labels = []
    
    for images, labels in loader:
        images, labels = images.to(DEVICE), labels.to(DEVICE)
        outputs = model(images)
        loss = criterion(outputs, labels)
        total_loss += loss.item()
        
        _, predicted = torch.max(outputs, 1)
        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())
    
    # 计算指标
    acc = np.mean(np.array(all_preds) == np.array(all_labels))
    balanced_acc = balanced_accuracy_score(all_labels, all_preds)
    avg_loss = total_loss / len(loader)
    
    return avg_loss, acc, balanced_acc

def train_model():
    best_val_acc = 0
    best_epoch = 0 
    for epoch in range(EPOCHS):
        # 训练阶段
        model.train()
        train_loss = 0.0
        train_preds = []
        train_labels = []
        
        progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{EPOCHS}")
        for images, labels in progress_bar:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            inputs, targets_a, targets_b, lam = cutmix_data(images, labels, alpha=0.3)
            
            outputs = model(inputs)
            loss = criterion(outputs, targets_a) * lam + criterion(outputs, targets_b) * (1. - lam)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            train_preds.extend(predicted.cpu().numpy())
            train_labels.extend(labels.cpu().numpy())
            
            progress_bar.set_postfix(loss=train_loss / len(progress_bar))
        
        # 计算训练集指标
        train_acc = np.mean(np.array(train_preds) == np.array(train_labels))
        train_balanced_acc = balanced_accuracy_score(train_labels, train_preds)
        train_loss = train_loss / len(train_loader)
        
        # 验证阶段
        val_loss, val_acc, val_balanced_acc = evaluate(val_loader)
        

        
        # 保存最佳模型
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            best_epoch = epoch
            torch.save(model.state_dict(), 'best_model.pth')
        
        scheduler.step(val_acc) 

        if (epoch - best_epoch) >= 10:  # 连续5个epoch无提升
            print(f"Early stopping triggered at epoch {epoch+1}, best epoch: {best_epoch+1}")
            break
                # 打印训练和验证指标
        print(f"\nEpoch {epoch+1}/{EPOCHS}:")
        print(f"Train - Loss: {train_loss:.4f}, Acc: {train_acc:.4f}, Balanced Acc: {train_balanced_acc:.4f}")
        print(f"Val   - Loss: {val_loss:.4f}, Acc: {val_acc:.4f}, Balanced Acc: {val_balanced_acc:.4f}")

In [14]:
!pip install ttach

### 预测函数 (^_^)

In [15]:
train_model()


Epoch 1/100: 100%|██████████| 82/82 [00:49<00:00,  1.66it/s, loss=3.48] 



Epoch 1/100:
Train - Loss: 3.4774, Acc: 0.1716, Balanced Acc: 0.1671
Val   - Loss: 1.3528, Acc: 0.6338, Balanced Acc: 0.6243


Epoch 2/100: 100%|██████████| 82/82 [00:36<00:00,  2.27it/s, loss=2.62] 



Epoch 2/100:
Train - Loss: 2.6201, Acc: 0.3455, Balanced Acc: 0.3443
Val   - Loss: 1.0517, Acc: 0.7108, Balanced Acc: 0.7081


Epoch 3/100: 100%|██████████| 82/82 [00:34<00:00,  2.36it/s, loss=2.45] 


...


Epoch 71/100:
Train - Loss: 1.1734, Acc: 0.6837, Balanced Acc: 0.6841
Val   - Loss: 0.2771, Acc: 0.9354, Balanced Acc: 0.9330


Epoch 72/100: 100%|██████████| 82/82 [00:32<00:00,  2.50it/s, loss=1.05]  



Epoch 72/100:
Train - Loss: 1.0484, Acc: 0.7661, Balanced Acc: 0.7660
Val   - Loss: 0.2778, Acc: 0.9354, Balanced Acc: 0.9327


Epoch 73/100: 100%|██████████| 82/82 [00:33<00:00,  2.48it/s, loss=1.15]  


Early stopping triggered at epoch 73, best epoch: 63


In [18]:
# 在导入库部分添加ttach
import ttach as tta

# 修改后的预测函数（保持原函数名称不变）
@torch.no_grad()
def predict():
    # 加载最佳模型
    model.load_state_dict(torch.load('best_model.pth'))
    model.eval()
    
    # 创建TTA包装模型（分类任务）
    tta_model = tta.ClassificationTTAWrapper(
        model,
        tta.aliases.five_crop_transform(crop_height=224, crop_width=224),
        merge_mode="mean"  # 几何平均效果更好
    )
    
    predictions = []
    for images, img_names in tqdm(test_loader, desc="TTA Predicting"):
        images = images.to(DEVICE)
        
        # 使用TTA模型预测
        outputs = tta_model(images)
        
        # 获取预测结果
        _, predicted = torch.max(outputs, 1)
        predicted_labels = [idx_to_label[idx.item()] for idx in predicted]
        predictions.extend(zip(img_names, predicted_labels))
    
    return predictions

# 保存函数无需修改
def save_predictions(predictions):
    submission_df = pd.DataFrame(predictions, columns=['filename', 'label'])
    submission_df.to_csv(SUBMISSION_FILE, index=False)

# 执行预测（保持调用方式不变）
predictions = predict()
save_predictions(predictions)
print(f"Submission file saved as {SUBMISSION_FILE}")


<ipython-input-18-77ab5f10ba84>:8: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('best_model.pth'))
TTA Predicting: 100%|██████████| 102/102

Submission file saved as /kaggle/working/submission.csv
